# Car Purchasement Classification
-------------------

In [1]:
# Imports
import pandas as pd
import seaborn as sns

## Data Analysis

In [2]:
# Cargamos el csv
df = pd.read_csv( "car_data.csv" )

#Visualizamos el dataset
df.head()

,User ID,Gender,Age,AnnualSalary,Purchased
0,385,Male,35,20000,0
1,681,Male,40,43500,0
2,353,Male,49,74000,0
3,895,Male,40,107500,1
4,661,Male,25,79000,0


In [3]:
#Visualizamos las dimensiones del dataset
df.shape

(1000, 5)

In [4]:
# Información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   User ID       1000 non-null   int64 
 1   Gender        1000 non-null   object
 2   Age           1000 non-null   int64 
 3   AnnualSalary  1000 non-null   int64 
 4   Purchased     1000 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 39.2+ KB


In [5]:
# Vemos las columnas que tiene nuestro dataset
df.columns

Index(['User ID', 'Gender', 'Age', 'AnnualSalary', 'Purchased'], dtype='object')

## Buscamos valores duplicados, null, outlier, etc (limpiamos el dataset)

* Valores duplicados

In [6]:
df.duplicated().sum() # No temos valores duplicados

0

* Valores faltantes (null)

In [7]:
df.isna().sum() # No temos valores faltantes

User ID         0
Gender          0
Age             0
AnnualSalary    0
Purchased       0
dtype: int64

## Descartaremos las columnas que no utilizaremos para realizar predicciones

In [8]:
df.columns

Index(['User ID', 'Gender', 'Age', 'AnnualSalary', 'Purchased'], dtype='object')

In [9]:
# Eliminamos la columna User ID porq no aporta nada a la predicción
df = df.drop( 'User ID', axis=1 )
df.columns

Index(['Gender', 'Age', 'AnnualSalary', 'Purchased'], dtype='object')

In [10]:
# Resumen estadístico
df.describe()

,Age,AnnualSalary,Purchased
count,1000.000000,1000.000000,1000.000000
mean,40.106000,72689.000000,0.402000
std,10.707073,34488.341867,0.490547
min,18.000000,15000.000000,0.000000
25%,32.000000,46375.000000,0.000000
50%,40.000000,72000.000000,0.000000
75%,48.000000,90000.000000,1.000000
max,63.000000,152500.000000,1.000000


In [11]:
# Vemos como se comporta la columna Gender
df['Gender'].value_counts()

Female    516
Male      484
Name: Gender, dtype: int64

In [12]:
# Vemos cuantas personas compraron
df['Purchased'].value_counts() # 402 Compraron

0    598
1    402
Name: Purchased, dtype: int64